# Creeps. Reading a .bib file, matching by name, and returning a message.


TODO

- the csv module will likely run on overleaf just fine: https://docs.python.org/3/library/csv.html

- create an environment for this project (BUT, we will try to call as few packages as possible).

- if someone's name is a pain in the arse, (i.e. not made with the characters a-z), what the heck do i do?

- find bib files that aren't generated by google scholar

- some author entries have double curly braces, i'm sure i'm stuffing them up: author={{World Food Summit}},


- just take names as given, don't edit. match. later on, can think about adding different options using TheFuzz and leivenstein distance stuff as well as dropping middle names and run:
    - if it's got a "." kill the "."
    - if it's got a word that is only 1 leter long, kill the word (that's gonna be an initial).
    - kill " Jr "; " Sr " .... any others like that?

In [72]:
import pandas as pd

# Take the database of offenders and return names. Turn this into a csv, just rip out what i want, so that later, i can do this without pandas.

df = pd.read_csv('asmd_incidents.csv', usecols=['Person', 'Institution', 'Original Link(s)']) # Just load the usable columns.
df = df[(df.Person != 'NAME UNKNOWN') & ~df.Person.isna()]

df['Person'] = df['Person'].astype(str).str.lower()
# df['Person'] = df['Person'].str.replace(',',"") # This is necessary for later on. There's one bloke who is so and so Junior, and has a comma in his name.
offenders = [offender for offender in list(df.Person)] # Offenders isn't used, so remove.

df.columns = ['person', 'institution', 'source']
df.to_csv('asmd.csv', index = False, sep = "\t") # Some of the names have a comma, and even some of URLs, so this helps for the string splits.

In [97]:
# Take the csv, using vanilla python, return a dictionary with names and links.
offenders, institutions, sources = [], [], []

# Open the CSV file in read mode
with open('asmd.csv', 'r') as file:
    for line in file.readlines():
        off, inst, src = line.split('\t')
#        off, inst, src = line.rstrip("\n").split('\t')
        offenders.append(off.strip())
        institutions.append(inst.strip())
        sources.append(src.strip())

In [98]:
# Take the csv, using vanilla python, return a dictionary with names and links.
offenders, institutions, sources = [], [], []

# Open the CSV file in read mode
# with open('asmd.csv', 'r') as file:
#     for line in file.readlines():
#         offenders.append(line.strip().split(',')[0])
#         institutions.append(line.strip().split(',')[1])
#         sources.append(line.strip().split(',')[2])


with open('asmd.csv', 'r') as file:
    for line in file.readlines():
        off, inst, src = line.split('\t')
#        off, inst, src = line.rstrip("\n").split('\t')
        offenders.append(off.strip())
        institutions.append(inst.strip())
        sources.append(src.strip())

# TODO - make it more memory efficient, and do this with the csv module, rather than saving first as lists.
# Create a dictionary of the above.
offd = dict(zip(offenders,zip(institutions, sources)))


# Take a .bib file, return the authors.
bibfile = "examples"
if not bibfile.endswith(".bib"):
    bibfile = bibfile + ".bib"

print(bibfile)

# TODO -- choose one way of doing this, probably the first way!
with open(bibfile) as refs:
    keys = [line.strip().split("{")[1].rstrip(',').strip().lower() for line in refs if line.strip().startswith("@")]

with open(bibfile) as refs:
    authors = [line.strip().split("{")[1].split("}")[0].split(" and ") for line in refs if line.strip().split("=")[0].lower().__contains__("author")]

# keys, authors = [], []
# with open('references.bib') as refs:
#     for line in refs.readlines():
#         if line.strip().startswith("@"):
#             keys.append(line.strip().split("{")[1].rstrip(',').strip())

#         elif line.strip().split("=")[0].lower().__contains__("author"):
#             authors.append(line.strip().split("{")[1].split("}")[0].split(" and "))



# If the name is of the form: "last_name, first_name (middle_names)", flip it around and remove the comma, so it is of the form: "first_name (middle names) last_name"
for i in range(len(authors)):
    authors[i] = [x.split(",")[1].strip().lower() + " " + x.split(",")[0].strip().lower() if x.__contains__(",") else x for x in authors[i]]

names = [item.lower() for sublist in authors for item in sublist]
# for i in range(len(keys)):
#    print(f'{keys[i]}: {authors[i]}')

examples.bib


In [99]:
# This works nicely only for exact name matching.
warnings = set([name for name in names if name in offenders])


# Turn to dictionary
bibd = dict(zip(keys, authors))


# Return the relevant bibliography identifiers for each offender.

article_ids, institutions, sources = [], [], [] # this overwrites previous stuff. ugly, i know.
for w in warnings:
    art_id = []
    for k, v in bibd.items():
        if w in v:
            art_id.append(k)

    article_ids.append(art_id)

    for k, v in offd.items():
        if k == w:
            institutions.append(offd.get(w)[0])
            sources.append(offd.get(w)[1])

article_ids


[['dittmann1976strindberg'], ['naqvi1995power', 'batra1987urban']]

In [88]:
for j in sources[i].split(";"):
    print(j)

https://www.usg.edu/assets/regents/documents/board_meetings/aug99min.pdf


In [102]:
# TODO turn the sources into a clickable link?

for i in range(len(warnings)):
    message = f"-- {list(warnings)[i]} [{institutions[i]}; {article_ids[i]}] is matched to the database of offenders. For details, see " 
    print(message)

    # Now, cycle through, print it if it's the final source in the list with a full stop after, otherwise print with a comma.
    for k, j in enumerate(sources[i].split(";")):
        if k != len(sources[i].split(";")) -1:
            details = f"\href{{{j}}}{{here}}, "
            print(details)
        else:
            details = f"\href{{{j}}}{{here}}."
            print(details)


    print('\n')    

-- reidar dittmann [St. Olaf College; ['dittmann1976strindberg']] is matched to the database of offenders. For details, see 
\href{https://www.insidehighered.com/news/2017/03/15/how-st-olaf-scrubbed-building-name-revered-professor-accused-sexual-misconduct}{here}
.


-- nadeem naqvi [University of Georgia; ['naqvi1995power', 'batra1987urban']] is matched to the database of offenders. For details, see 
\href{https://www.usg.edu/assets/regents/documents/board_meetings/aug99min.pdf}{here}
.




In [96]:
# Cleanish for overleaf.


# Take the csv, using vanilla python, return a dictionary with names and links.
offenders, institutions, sources = [], [], []

# Open the CSV file in read mode
with open('asmd.csv', 'r') as file:
    for line in file.readlines():
        offenders.append(line.strip().split(',')[0])
        institutions.append(line.strip().split(',')[1])
        sources.append(line.strip().split(',')[2])

# TODO - make it more memory efficient, and do this with the csv module, rather than saving first as lists.
# Create a dictionary of the above.
offd = dict(zip(offenders,zip(institutions, sources)))


# Take a .bib file, return the authors.
bibfile = "examples"
if not bibfile.endswith(".bib"):
    bibfile = bibfile + ".bib"

# TODO -- choose one way of doing this, probably the first way!
with open(bibfile) as refs:
    keys = [line.strip().split("{")[1].rstrip(',').strip().lower() for line in refs if line.strip().startswith("@")]

with open(bibfile) as refs:
    authors = [line.strip().split("{")[1].split("}")[0].split(" and ") for line in refs if line.strip().split("=")[0].lower().__contains__("author")]


# If the name is of the form: "last_name, first_name (middle_names)", flip it around and remove the comma, so it is of the form: "first_name (middle names) last_name"
for i in range(len(authors)):
    authors[i] = [x.split(",")[1].strip().lower() + " " + x.split(",")[0].strip().lower() if x.__contains__(",") else x for x in authors[i]]

names = [item.lower() for sublist in authors for item in sublist]


# This works nicely only for exact name matching.
warnings = set([name for name in names if name in offenders])


# Turn to dictionary
bibd = dict(zip(keys, authors))


# Return the relevant bibliography identifiers for each offender.
article_ids, institutions, sources = [], [], [] # this overwrites previous stuff. ugly, i know.
for w in warnings:
    art_id = []
    for k, v in bibd.items():
        if w in v:
            art_id.append(k)

    article_ids.append(art_id)

    for k, v in offd.items():
        if k == w:
            institutions.append(offd.get(w)[0])
            sources.append(offd.get(w)[1])


# # TODO turn the sources into a clickable link?
# for i in range(len(warnings)):
#     message = f"{list(warnings)[i]} has been found in the database of offenders. They're authors in {article_ids[i]} and the database has them working at {institutions[i]}. For further details, see {sources[i]}."
#     print(message)


for i in range(len(warnings)):
    message = f"-- {list(warnings)[i]} [{institutions[i]}; {article_ids[i]}] is matched to the database of offenders. For details, see " 
    print(message)
    for j, k in enumerate(sources[i].split(";")):
        details = f"\href{{{j}}}{{here}}"
        print(details)
        if j != len(sources[i].split(";")) -1:
            print(", ")
        else:
            print(".")
    print('\n')


IndexError: list index out of range

In [95]:
len(sources[i].split(";"))

1